In [184]:
import numpy as np
import pandas as pd
from main import get_embedding
from torch.nn.functional import one_hot
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from train import MyDataset, train
from classification_head import ClassificationHead
from torch.utils.data import DataLoader

In [185]:
df = pd.read_csv("../Creating dataset/oss.csv")
df.head()

,content,Class
0,Tap to see more options,Normal
1,Abishek Prasad and 1 other reacted to your post.,Normal
2,thebarefootsprinter is going live now with wil...,Critical
3,Checking for new messages,Normal
4,2 new messages,Normal


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1383 entries, 0 to 1382
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  1383 non-null   object
 1   Class    1354 non-null   object
dtypes: object(2)
memory usage: 21.7+ KB


In [187]:
df.Class.unique()

array(['Normal', 'Critical', 'Low Priority', 'Important', nan],
      dtype=object)

In [188]:
df.isna().sum()

content     0
Class      29
dtype: int64

In [189]:
df = df.dropna()

In [190]:
df.isna().sum()

content    0
Class      0
dtype: int64

In [191]:
df.Class.unique()

array(['Normal', 'Critical', 'Low Priority', 'Important'], dtype=object)

In [236]:
class_map = {'Normal' : 0 , 'Critical' : 1, 'Low Priority' : 2, 'Important': 3}
i_c = {v: k for k, v in class_map.items()}


In [193]:
df.Class = df.Class.map(class_map)

In [194]:
df.head()

,content,Class
0,Tap to see more options,0
1,Abishek Prasad and 1 other reacted to your post.,0
2,thebarefootsprinter is going live now with wil...,1
3,Checking for new messages,0
4,2 new messages,0


In [195]:
def embed(text):
    return get_embedding(str(text))

In [196]:
df['Embedding'] = df.Class.map(embed)

In [197]:
df['Class_OH'] = [one_hot(torch.tensor([c]), num_classes=len(class_map)).squeeze().numpy() for c in df.Class]

In [198]:
df.head()

,content,Class,Embedding,Class_OH
0,Tap to see more options,0,"[0.20385437334577242, 0.3925517400105794, -0.7...","[1, 0, 0, 0]"
1,Abishek Prasad and 1 other reacted to your post.,0,"[0.20385437334577242, 0.3925517400105794, -0.7...","[1, 0, 0, 0]"
2,thebarefootsprinter is going live now with wil...,1,"[-0.3952137877543767, 0.006723076105117798, -0...","[0, 1, 0, 0]"
3,Checking for new messages,0,"[0.20385437334577242, 0.3925517400105794, -0.7...","[1, 0, 0, 0]"
4,2 new messages,0,"[0.20385437334577242, 0.3925517400105794, -0.7...","[1, 0, 0, 0]"


In [199]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [200]:
train_dataset = MyDataset(x=train_df.Embedding.tolist(), y=train_df.Class.tolist())
test_dataset = MyDataset(x=test_df.Embedding.tolist(), y=test_df.Class.tolist())

In [201]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [202]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [217]:
Classification = ClassificationHead(in_fet=384, out_fet=4, p=.4).to(device).double()

In [218]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Classification.parameters(), lr=0.001)
train(epochs=10, model=Classification, optimizer=optimizer, loss_fn=loss_fn, dataset_train=train_loader, dataset_test=test_loader)

Epoch: 0, Train Loss: 0.3551104023121386, Test Loss: 0.030032097897234378
Epoch: 1, Train Loss: 0.00820196956297295, Test Loss: 0.00040058229845161745
Epoch: 2, Train Loss: 0.0005385766332151177, Test Loss: 3.693063426713003e-05
Epoch: 3, Train Loss: 0.00012899956172573733, Test Loss: 3.4012425897669276e-05
Epoch: 4, Train Loss: 0.00010187309873843982, Test Loss: 1.2588656340544285e-05
Epoch: 5, Train Loss: 4.3689707922562654e-05, Test Loss: 7.82039363605754e-06
Epoch: 6, Train Loss: 5.601149136610623e-05, Test Loss: 4.956205695640458e-06
Epoch: 7, Train Loss: 2.7356147081432954e-05, Test Loss: 3.733989774751091e-06
Epoch: 8, Train Loss: 2.6360081667374918e-05, Test Loss: 2.7396618834604608e-06
Epoch: 9, Train Loss: 1.77251257581096e-05, Test Loss: 2.100851906431123e-06


In [219]:
correct = 0
total = 0
Classification.eval()
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device).double()
        y = y.to(device)
        outputs = Classification(x)
        _, predicted = torch.max(outputs, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 1.0000


In [237]:
def infer(text):
    Classification.eval()
    with torch.no_grad():

        text_embeddings = get_embedding(text)
        predicton = Classification(torch.tensor(text_embeddings).to(device))
        cato = i_c.get(int(torch.argmax(predicton)))
        return cato

In [238]:
infer("you are fired")

'Critical'